In [1]:
import os
# save using vs code
SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'
# Download Kafka Jar file, this for readStream.format("kafka"), "kafka" is a driver
# kafka driver code is part of Maven Jar file
# https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/3.1.3
# pyspark-shell shall download the jar file behind..
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'

In [2]:
import sys
sys.stdout = open("/dev/stdout", "w")
print("Now my output shall be on terminal, not on my notebook")

Now my output shall be on terminal, not on my notebook
:: loading settings :: url = jar:file:/opt/spark-3.1.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)

root
 |-- value: struct (nullable = true)
 |    |-- symbol: string (nullable = true)
 |    |-- price: double (nullable = true)
 |    |-- volume: long (nullable = true)
 |    |-- timestamp: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)

root
 |-- symbol: string (nullable = true)
 |-- price: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- timestamp: long (nullable = true)

root
 |-- symbol: string (nu

In [3]:
# here we implement windowed candle data for 1 minute
# here we implement windowed candle data for 3 minute
# here we implement windowed candle data for 5 minute

# we use hadoop-vm as hostname, which is running on another system


#     kafka-topics  --create --bootstrap-server hadoop-vm:9092 --replication-factor 1 --partitions 1 --topic stock-ticks 
    
#     kafka-topics  --create --bootstrap-server hadoop-vm:9092 --replication-factor 1 --partitions 1 --topic candles-1min 
#     kafka-console-consumer --bootstrap-server hadoop-vm:9092 --topic  candles-1min  --from-beginning
 
#     kafka-topics  --create --bootstrap-server hadoop-vm:9092 --replication-factor 1 --partitions 1 --topic candles-3min 
#     kafka-console-consumer --bootstrap-server hadoop-vm:9092 --topic  candles-3min  --from-beginning

#     kafka-topics  --create --bootstrap-server hadoop-vm:9092 --replication-factor 1 --partitions 1 --topic candles-5min 
#     kafka-console-consumer --bootstrap-server hadoop-vm:9092 --topic  candles-5min  --from-beginning

In [4]:
import findspark
findspark.init()

In [5]:
import pyspark


from pyspark.sql import SparkSession
# spark groupBy has default setting for spark.sql.shuffle.partitions as 200
# we set to  4, should NOT be done in production 
spark = SparkSession.builder.master("local[1]")\
                            .config('spark.sql.shuffle.partitions', 4)\
                            .appName("SparkStreamingKafkaCandles").getOrCreate()

22/03/27 00:09:36 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/27 00:09:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f1017639-4d2f-4400-80cb-9020a406ccb3;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.3 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central


In [6]:
# read from kafka, here spark is consumer for kafka topic called test
# spark streaming works as dataframe/sql
# group.id is consumer group id
# subcribe is kafka topic
# "kafka" driver is not available by default with spark, we need to download it, we did on cell 1
# GROUP ID SHOULD BE CHANGED FOR EVERY PERSON 
kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "hadoop-vm:9092")\
  .option("subscribe", "stock-ticks")\
  .option("group.id", "stock-ticks-group23")\
  .load()

In [7]:
# key and value are binary type, we need to CAST To STRING type
kafkaDf.printSchema()
# timestampType values
# CreateTime:  Timestamp relates to message creation time as set by a Kafka client/producer
# LogAppendTime : Timestamp relates to the time a message was appended to a Kafka log.
# UnknownType

In [8]:
# Kafka value, key are in binary format
# we need to type cast to STRING, 
# we pick only value, timestamp as we need for 10 minute window
ticksDf = kafkaDf.selectExpr("CAST(value AS STRING)", "timestamp")
ticksDf.printSchema() # we get only value as string

In [9]:
# echoOnconsole = ticksDf\
#                 .writeStream\
#                 .outputMode("update")\
#                 .format("console")\
#                 .option("truncate", False)\
#                 .start() # start the query. spark will subscribe for data

In [10]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, DoubleType, StringType, LongType, TimestampType

schema = StructType( [
    StructField("symbol", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("volume", LongType(), True),
    StructField("timestamp", LongType(),  True)
])

In [11]:
# we parse value which is JSON string to kafka struct, 
# this create a new column [replacing existing value which is string] with type
jsonDf = ticksDf.withColumn("value", F.from_json("value", schema))
jsonDf.printSchema()

In [12]:
# now extract all the sub fields symbol, price, volume, timestamp into data frame
# exisitng value, timestamp shall be removed. existing timestamp is value produced time, not a stock tick time
# value.timestamp that comes from producer is basically a event time
stockTickDf = jsonDf.select (F.col("value.*")) # extract all sub fields as schema
stockTickDf.printSchema()

In [13]:
# we have data coming in stream, each record has timestamp
# we have milli seconds 1647283268253, the actual seconds Mon Mar 14 2022 14:41:08
# now we have to calculate 1 minute window, sum(volume) up to 1 minute
# after conversions, we take timestamp which is truncatd to minute instead of milli seconds, drop temp columns
stockTickDf = stockTickDf\
                .withColumn("timestamp", (F.col("timestamp") / 1000).cast("timestamp"))\
                

stockTickDf.printSchema()
# echoOnconsole = stockTickDf\
#                 .writeStream\
#                 .outputMode("update")\
#                 .format("console")\
#                 .option("truncate", False)\
#                 .start() # start the query. spark will subscribe for data

In [14]:
# aggregate
# sum (volume), sum(traded_value), candles open, high, low, close [OHLC]
# groupBy by symbol, timestamp

# now add delayed record processing with waterMark
stockTickDf1Min = stockTickDf\
                            .selectExpr("timestamp", "symbol",  "struct(*) as value")\
                            .withWatermark("timestamp", "1 minutes")\
                            .groupBy("symbol", F.window("timestamp", "60 seconds"))\
                          .agg( F.collect_list("value").alias("ticks"))

stockTickDf1Min.printSchema()

In [ ]:
# spark stream with batch output
# spark basically process the data as batch
# for developer convinence, we have api, that can give us finite data frame for writing to file/jsbc/db/custom coding
# process discrete output batch
# this function is called for every update/delete/comlete triggers with result data set as dataframe
# candleBatchDf is BATCH DATAFRAME
# stockTickDf1Min is STREAM DATAFRAME
def processBatchData(candleBatchDf, batch_id):
    from pyspark.sql.functions import explode, col

    print ("process batch called", batch_id, "writing ", candleBatchDf.count())
    #result = candleBatchDf.collect()
    #print(result)
    #candleBatchDf.printSchema()
    #candleBatchDf.show(2, truncate = False)
    
    # pick all the elements inside ticks array into element
    # breaking group using explode
    candleTickBatchDf = (candleBatchDf.select("window", explode(F.column("ticks")) )
                                      .select(F.col("window.*"), F.col("col.*"))
                        )
    
    candleTickBatchDf.printSchema()
    candleTickBatchDf.show(2, truncate=False)
    # write to csv/json/parqeut/database /jdbc etc
    # window is nested column, has its children columns called start and end
    # start and end are part of nested window column
    # append mode will not delete existin data, instead append to existing table
     
    
stockTickDf1Min.writeStream.outputMode("append").foreachBatch(processBatchData).start()
    
# now go to mysql, check candle_1min exist
# show tables;
# select * from candle_1min

In [ ]:
# write result to kafka
# we need to publish all the field together as value
# we take all columns, convert to json string
# publish json as value to kafka

stockTickDf1MinKafka = stockTickDf1Min\
                            .selectExpr("to_json(struct(*)) AS value")

stockTickDf1MinKafka\
            .writeStream\
            .format("kafka")\
            .outputMode("update")\
            .option("kafka.bootstrap.servers", "localhost:9092")\
            .option("topic", "candles-1min")\
            .option("checkpointLocation", "file:///tmp/spark3")\
            .start()